In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Changing directory to the directory containing dataset
%cd drive/MyDrive/NNFL data/Data_A2/

/content/drive/MyDrive/NNFL data/Data_A2


In [ ]:
%ls -l

total 87
-rw------- 1 root root   259 Apr 29 07:23 class_label.mat
-rw------- 1 root root 70617 Apr 22 07:54 data55.xlsx
-rw------- 1 root root 17039 Apr 29 07:25 data5.xlsx


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from pprint import pprint

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def sigmoidFuntion(Z):
    return 1/(1+np.exp(-Z)),Z

def relu(Z):
    A = np.maximum(0,Z)
    return (Z,A)

def reluBack(dA, cache):
    Z = cache
    
    dZ = np.array(dA, copy=True) 
    dZ[Z <= 0] = 0
    
    return dZ

def sigmoidBack(dA, cache):
    Z = cache
    
    s = 1/(1+np.exp(-Z))
    dZ = dA * s * (1-s)

    return dZ

def initializeParams(layerDimensions, para, stack):
    parameters = {}
    L = len(layerDimensions)            
    for l in range(1, L-1):
        if(stack==False):
            parameters['W' + str(l)] = np.random.randn(layerDimensions[l], layerDimensions[l-1])
        else:
            parameters['W' + str(l)] = para[l-1]['W1']
        parameters['b' + str(l)] = np.zeros((layerDimensions[l], 1))
    parameters['W' + str(L-1)] = np.random.randn(layerDimensions[L-1], layerDimensions[L-2])
    parameters['b' + str(L-1)] = np.zeros((layerDimensions[L-1],1))
    return parameters

def linearForward(A, W, bias):
    Z = np.dot(W, A) + bias
    cache = (A, W, bias)
    return Z, cache

def linearActivationForward(A_prev, W, bias, activation):
    
    if(activation == "sigmoid"):
        Z, linear_cache = linearForward(A_prev, W, bias)
        A, activation_cache = sigmoidFuntion(Z)
    
    elif(activation == "relu"):
        Z, linear_cache = linearForward(A_prev, W, bias)
        A, activation_cache = relu(Z)
        
    cache = (linear_cache, activation_cache)
    
    return A, cache

def forwardModelL(X, params):
    caches = []
    A = X
    L = (len(params) //2)
    for l in range(1, L):
        A_prev = A 
        A, cache = linearActivationForward(A, params['W'+str(l)], params['b'+str(l)], "sigmoid")
        caches.append(cache)
    
    AL, cache = linearActivationForward(A, params['W'+str(L)], params['b'+str(L)], "sigmoid")
    caches.append(cache)

    return AL,caches

def costComputation(AL, Y):
    m = Y.shape[1]
    cost = -(1/m)*np.sum(Y*np.log(AL)+(1-Y)*np.log(1-AL))
    cost = np.squeeze(cost)
    return cost

def costComputationAutoencoder(AL, Y, parameters):
    m = Y.shape[1]
    cost = (1/(2*m))*np.sum(np.linalg.norm(AL-Y))
    L = (len(parameters) //2)
    
    return cost

def linearBack(dZ, cache):
    A_prev, W, _ = cache
    _, m = A_prev.shape

    dW = (1/m)*np.dot(dZ, A_prev.T)
    db = (1/m)*np.sum(dZ, axis=1, keepdims=True)
    dAP = np.dot(W.T, dZ)
   
    return dAP, dW, db

def linearActivationBack(dA, cache, activation):
    linearCache, activationCache = cache
    
    if (activation == "relu"):
        dZ = reluBack(dA, activationCache)
        dAP, dW, db = linearBack(dZ, linearCache)
        
    elif (activation == "sigmoid"):
        dZ = sigmoidBack(dA, activationCache)
        dAP, dW, db = linearBack(dZ, linearCache)
    
    return dAP, dW, db

def modelLBack(AL, Y, caches):
    gradients = {}
    L = len(caches) 
    m = AL.shape[1]
    Y = Y.reshape(AL.shape) 
    
    dAL = -(np.divide(Y,AL)-np.divide(1-Y,1-AL))
    
    current_cache = caches[L-1]
    gradients["dA" + str(L-1)], gradients["dW" + str(L)], gradients["db" + str(L)] = linearActivationBack(dAL, current_cache, "sigmoid")
   
    for l in reversed(range(L-1)):
    
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = linearActivationBack(gradients["dA"+str(l+1)],current_cache,"sigmoid")
        gradients["dA" + str(l)] = dA_prev_temp
        gradients["dW" + str(l + 1)] = dW_temp
        gradients["db" + str(l + 1)] = db_temp

    return gradients

def updateParams(parameters, gradients, learning_rate):
   
    L = (len(parameters) //2)

    for l in range(L):
        parameters["W" + str(l+1)] = parameters["W" + str(l+1)]-learning_rate*grads["dW"+str(l+1)]
        parameters["b" + str(l+1)] = parameters["b" + str(l+1)]-learning_rate*grads["db"+str(l+1)]
  
    return parameters

def layerLModel(X, Y, layers_dims, num_iterations,stack, learning_rate = 0.1):

    costs = []                        
 
    parameters = initializeParams(layers_dims,para,stack)

    for i in range(0, num_iterations):

        AL, caches = forwardModelL(X,parameters)
        
        if(stack==True):
            cost = costComputation(AL, Y)
        else:
            cost = costComputationAutoencoder(AL, Y, parameters)
        
        grads = modelLBack(AL,Y,caches)
     
        parameters = updateParams(parameters,grads,learning_rate)

        if(i%100==0):
            print ("Cost after iteration %i: %f" %(i, cost))
        costs.append(cost)
    print(cost)
                
    return parameters, costs

In [ ]:
dataset = pd.read_excel('/content/drive/MyDrive/NNFL data/Data_A2/data55.xlsx')

row, col = dataset.shape
feats = col - 1 

# normalization
dataset.loc[:, dataset.columns != feats] = (dataset.loc[:, dataset.columns != feats]-dataset.loc[:, dataset.columns != feats].mean(axis=0))/dataset.loc[:, dataset.columns != feats].std(axis=0)

# spliting dataset into train test and val
training_data, validation_data, testing_data = np.split(dataset.sample(frac=1),[int(0.7*len(dataset)), int(0.8*len(dataset))])

training_data = np.array(training_data)
validation_data = np.array(validation_data)
testing_data = np.array(testing_data)
training_data_X = training_data[:, :feats]
training_data_y = training_data[:, feats]
validation_data_X = validation_data[:, :feats]
validation_data_y = validation_data[:, feats]
testing_data_X = testing_data[:, :feats]
testing_data_y = testing_data[:, feats]

train_row, train_col = training_data_X.shape


para = []
lr= 0.005

layerDemensions = [72,64,72]
params, costs = layerLModel(training_data_X, training_data_y, layerDemensions, num_iterations=1000, stack = False, learning_rate=lr)
paramsAE1 = params

W1=paramsAE1['W1']
b1=paramsAE1['b1']
X_new,_=linearActivationForward(X.T,W1,b1,"sigmoid")
X_new.shape


layerDemensions = [64,16,64]
params, costs = layerLModel(training_data_X, training_data_y, layerDemensions,num_iterations=1000,stack = False,learning_rate=lr)
paramsAE2 = params

W1 = paramsAE2['W1']
b1 = paramsAE2['b1']
X_new, _ =  linearActivationForward(X.T, W1, b1, "sigmoid")


layerDemensions = [16,4,16]
params, costs = layerLModel(training_data_X, training_data_y, layerDemensions, learning_rate=lr, num_iterations=1000, stack=False)
paramsAE3= params


para=[paramsCopy, paramsAE2, paramsAE3]

layerDemensions = [72,64,16,4,1]
params, costs = layerLModel(training_data_X, training_data_y, layerDemensions,num_iterations=10000, stack=True, learning_rate=0.125)

p, l = forwardModelL(training_data_X, params)
p = (p>0.5).astype(int)
a = training_data_y-p
a = np.sum((a!=0).astype(int))
accuracy = (p.shape[1]-a)/p.shape[1]
print('Testing Accuracy: ')
metrics(p, data_testing_y)
print()
print('Validation Accuracy: ')
metrics(p, data_validation_y)